# Clustering

In [1]:
import ais_processing
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiLineString

In [2]:
df = gpd.read_file('AIS_2015_01_SF_Bay_transits.csv', GEOM_POSSIBLE_NAMES='geometry', KEEP_GEOM_COLUMNS='NO')

In [3]:
df.head()

,start_DTG,end_DTG,SOG,LOA_unbinned,day,month,length,max_SOG,LOA,geometry
0,2015-01-19 18:21:43,2015-01-20 00:03:48,47.7,28.65,Mon,Jan,7.900541642041586,40-50kt,0-100m,"LINESTRING (-122.38206 37.74708, -122.38202 37..."
1,2015-01-11 18:39:55,2015-01-12 04:13:36,11.3,27.42,Sun,Jan,1.5502465242070502,10-20kt,0-100m,"LINESTRING (-121.90111 38.04800, -121.90470 38..."
2,2015-01-16 09:12:44,2015-01-16 20:13:05,13.0,27.42,Fri,Jan,1.5471122663391763,10-20kt,0-100m,"LINESTRING (-121.90003 38.04878, -121.90000 38..."
3,2015-01-09 05:25:13,2015-01-09 15:02:24,9.9,27.42,Fri,Jan,1.5416339346080299,0-10kt,0-100m,"LINESTRING (-121.90071 38.04712, -121.90374 38..."
4,2015-01-10 05:47:36,2015-01-10 15:15:06,10.9,27.42,Sat,Jan,1.5387565264193501,10-20kt,0-100m,"LINESTRING (-121.90134 38.04802, -121.90414 38..."


In [4]:
transits = ais_processing.cluster_transits(df, hausdorff_threshold=0.035)

#### Build a test *non-production* gpd.GeoDataFrame that captures the clustered routes to test performance on the site

In [5]:
max_SOG  = [df.max_SOG[transit].reset_index(drop=True)[0] for transit in transits]
LOA      = [df.LOA[transit].reset_index(drop=True)[0] for transit in transits]
geometry = [df.geometry[transit].reset_index(drop=True)[0] for transit in transits]

In [6]:
pd.DataFrame.from_dict({'max_SOG': max_SOG, 'LOA': LOA, 'geometry': geometry})

,max_SOG,LOA,geometry
0,40-50kt,0-100m,"LINESTRING (-122.38206 37.74708, -122.38202 37..."
1,10-20kt,0-100m,"LINESTRING (-121.90111 38.048, -121.9047 38.04..."
2,10-20kt,0-100m,"LINESTRING (-121.90003 38.04878, -121.9 38.049..."
3,10-20kt,0-100m,"LINESTRING (-121.90223 38.04816, -121.90574 38..."
4,10-20kt,0-100m,"LINESTRING (-121.90381 38.04698, -121.90739 38..."
...,...,...,...
1484,0-10kt,0-100m,"LINESTRING (-122.38545 37.74738, -122.38563 37..."
1485,0-10kt,0-100m,"LINESTRING (-122.2113 37.59956, -122.21117 37...."
1486,0-10kt,0-100m,"LINESTRING (-122.37312 37.90554, -122.3731 37...."
1487,0-10kt,0-100m,"LINESTRING (-122.39437 37.90685999999999, -122..."


In [7]:
vessels = gpd.GeoDataFrame.from_dict({'Speed': max_SOG, 'Length': LOA, 'Geometry': geometry})

In [8]:
vessels.to_csv('bay_traffic.csv')